In [9]:
import os
import sqlite3
import pandas as pd
from collections import defaultdict
import pyarrow

def aggregate_totals(folder_path, keywords):
    subject_totals = defaultdict(int)
    
    db_files = [f for f in os.listdir(folder_path) if f.endswith(".db")]
    
    for db_file in db_files:
        db_path = os.path.join(folder_path, db_file)
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        for kw in keywords:
            cursor.execute("""
                SELECT ns.subjectId, n.total 
                FROM normqueries n
                JOIN normquery_subjects ns ON n.normquery_id = ns.normquery_id
                WHERE n.kw = ?
            """, (kw,))
            
            for subjectId, total in cursor.fetchall():
                subject_totals[subjectId] += total
        
        conn.close()
    
    return subject_totals

def save_to_csv(subject_totals, output_file):
    df = pd.DataFrame(list(subject_totals.items()), columns=["subjectId", "total"])
    df.to_csv(output_file, index=False)





In [2]:
folder_path = "/home/i-b8o/Downloads/work"
keywords = [
"пупырчатая пленка"
"пупырка для упаковки",
"пупырка",
"пленка пупырчатая для упаковки",
"пупырчатая пленка для упаковки",
"пузырчатая пленка",
"пленка пупырчатая",
"упаковочная пленка с пузырьками",
"пупырка для упаковки рулон",
"пузырчатая пленка для упаковки",
"пленка для обертывания с пузырьками",
"пленка упаковочная пупырчатая",
"пупырчатая пленка трехслойная",
"пленка пупырчатая для упаковки 10 м",
"пленка для обертывания чемодана",
"пленка пупырчатая для упаковки 50 м",
"пупырышки",
"упаковка пупырчатая пленка",
"пленка пузырчатая",
"пленка для упаковки пупырчатая",
"пленка пупырчатая для переезда",
"пленка пупырка",
"пупырчатая пленка для переезда",
"пупырка для упаковки трехслойная",
"пленка с пупырышками упаковочная",
"пленка для обертывания мебели широкая",
"пленка для чемодана упаковочная",
"пупырышки для упаковки",
"пупырки",
"пупырчатая плёнка",
"пакеты из пупырчатой пленки",
"рулон пупырчатой пленки",
"пузырьковая пленка",
"пупырчатая пленка для упаковки трехслойная"

]

result = aggregate_totals(folder_path, keywords)
    

In [4]:
def merge_with_categories(subject_totals, categories_file, output_file):
    df_totals = pd.DataFrame(list(subject_totals.items()), columns=["subjectId", "total"])
    df_categories = pd.read_parquet(categories_file)
    
    df_merged = df_totals.merge(df_categories, on="subjectId", how="left")
    df_merged.to_csv(output_file, index=False)



In [6]:
categories_file = "/home/i-b8o/Downloads/done/categories.parquet"  # Укажите путь к файлу категорий

result

defaultdict(int,
            {6466: 1023097,
             2747: 1024854,
             3676: 1486331,
             453: 1473227,
             3756: 174308,
             7311: 4746,
             1288: 465,
             2101: 2837,
             2552: 1356,
             1536: 6306,
             390: 6306,
             2825: 6306,
             1291: 6306,
             267: 6306,
             4751: 6306,
             24: 6306,
             3742: 6306,
             1056: 6306,
             297: 6306,
             4265: 6306,
             4143: 6306,
             2865: 11652,
             947: 11652,
             957: 6306,
             3905: 6306,
             336: 6306,
             3924: 6306,
             3797: 6306,
             731: 6306,
             1757: 6306,
             2915: 11652,
             1904: 6306,
             2547: 118986,
             4599: 6306,
             120: 6306,
             381: 11652,
             7202: 591,
             7641: 2372,
             2783: 591,
   

In [10]:
merge_with_categories(result, categories_file, "subject_totals_with_names.csv")

In [13]:
import string

BASE62_ALPHABET = string.digits + string.ascii_letters

def string_to_number(s):
    """Преобразует строку в компактное число с помощью base62."""
    num = int.from_bytes(s.encode(), 'big')  # Преобразуем строку в большое число
    return encode_base62(num)

def number_to_string(encoded_str):
    """Преобразует base62 обратно в строку."""
    num = decode_base62(encoded_str)  # Декодируем base62 в число
    return num.to_bytes((num.bit_length() + 7) // 8, 'big').decode()

def encode_base62(num):
    """Преобразует число в base62."""
    if num == 0:
        return BASE62_ALPHABET[0]
    base62 = []
    while num:
        num, rem = divmod(num, 62)
        base62.append(BASE62_ALPHABET[rem])
    return ''.join(reversed(base62))

def decode_base62(base62_str):
    """Преобразует base62 обратно в число."""
    num = 0
    for char in base62_str:
        num = num * 62 + BASE62_ALPHABET.index(char)
    return num

# Пример
original_string = "paket-iz-vpp-6*20-sm"
encoded_number = string_to_number(original_string)
decoded_string = number_to_string(encoded_number)

print("Base62:", encoded_number)
print("Восстановленная строка:", decoded_string)


Base62: g2acdqJxLcnzj9uUKVvuLUoFHw9
Восстановленная строка: paket-iz-vpp-6*20-sm
